<a href="https://colab.research.google.com/github/lucasuff/Vies-de-genero-em-PLN-em-portugues/blob/main/ETW2VM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install transformers

     |████████████████████████████████| 3.4 MB 5.0 MB/s 
     |████████████████████████████████| 3.3 MB 43.5 MB/s 
     |████████████████████████████████| 596 kB 55.0 MB/s 
     |████████████████████████████████| 895 kB 64.2 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizer, BertModel
import torch

In [ ]:
model = BertModel.from_pretrained('neuralmind/bert-base-portuguese-cased', output_hidden_states = True)
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False, never_split=['lol'])

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

In [ ]:
def generate_embeddings(text):
  try:
    # Add the special tokens.
    marked_text = "[CLS] " + text + " [SEP]"

    # Split the sentence into tokens.
    tokenizer.add_tokens('lol')
    tokenized_text = tokenizer.basic_tokenizer.tokenize(marked_text)

    # Map the token strings to their vocabulary indeces.
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    # Mark each of the 22 tokens as belonging to sentence "1".
    segments_ids = [1] * len(tokenized_text)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    # Run the text through BERT, and collect all of the hidden states produced
    # from all 12 layers. 
    with torch.no_grad():
      outputs = model(tokens_tensor, segments_tensors)
      # Evaluating the model will return a different number of objects based on 
      # how it's  configured in the `from_pretrained` call earlier. In this case, 
      # becase we set `output_hidden_states = True`, the third item will be the 
      # hidden states from all layers. See the documentation for more details:
      # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
      hidden_states = outputs[2]
      
    # Concatenate the tensors for all layers. We use `stack` here to
    # create a new dimension in the tensor.
    token_embeddings = torch.stack(hidden_states, dim=0)

    # Remove dimension 1, the "batches".
    token_embeddings = torch.squeeze(token_embeddings, dim=1)

    # Swap dimensions 0 and 1.
    token_embeddings = token_embeddings.permute(1,0,2)

    # Stores the token vectors, with shape [22 x 768]
    token_vecs_sum = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Sum the vectors from the last four layers.
        sum_vec = torch.sum(token[-4:], dim=0)
        
        # Use `sum_vec` to represent `token`.
        token_vecs_sum.append(sum_vec)
      
    return [tokenized_text[3]], [token_vecs_sum[3]]
  except Exception as e:
    print(e)
    return None, None




In [ ]:
import json
import re

file = open("/content/drive/MyDrive/curso_word2vec/contents_woman.json", "r")
trip_contents = json.load(file)
file.close()

In [ ]:
vocabulary = json.load(open("/content/drive/MyDrive/curso_word2vec/vocabulary.json", "r"))

In [ ]:
tokenized_text = []
token_vecs_sum = []

In [ ]:
for word in vocabulary:
    temp_tokenized_text, temp_token_vecs_sum = generate_embeddings(word)
    if temp_tokenized_text and temp_token_vecs_sum:
      tokenized_text += temp_tokenized_text
      token_vecs_sum += temp_token_vecs_sum

index out of range in self


In [ ]:
def get_word_vector(word):
  for i in range(0, len(tokenized_text)):
    if tokenized_text[i]==word:
      return token_vecs_sum[i]


In [ ]:
def print_closest_words(pos1, pos2=None, neg=None, n=5):
    vec = get_word_vector(pos1)
    if pos2:
      vec = vec + get_word_vector(pos2)
    if neg:
      vec = vec - get_word_vector(neg)
    dists = torch.norm(torch.stack(token_vecs_sum) - vec, dim=1)     # compute distances to all words
    lst = sorted(enumerate(dists.numpy()), key=lambda x: x[1]) # sort by distance
    results = {}
    for idx, difference in lst[1:n+1]:                         # take the top n
        if tokenized_text[idx]!=pos2 and tokenized_text[idx]!=pos1 and tokenized_text[idx]!=neg and tokenized_text[idx] not in results:
          results[tokenized_text[idx]] = difference

    for key in results:
      print(key, results[key])

In [ ]:
print_closest_words("mulher", "rei", "homem")

reis 51.93973
monarca 55.21746
príncipe 55.297752
princesa 55.395176


In [ ]:
print_closest_words("homem", "rainha", "mulher")

príncipe 48.71744


In [ ]:
print_closest_words("homem", "chefe", "mulher")

In [ ]:
print_closest_words("mulher", "chefe", "homem")

chefes 53.984318
líder 55.060165
secretária 57.1857
esposa 57.60809


In [ ]:
print_closest_words("homem", "professora", "mulher")

In [ ]:
print_closest_words("mulher", "professor", "homem")

professora 47.030415
professores 53.145233
docente 55.439556


In [ ]:
print_closest_words("mulher", "médico", "homem")

médica 45.98616
médicas 51.87478


In [ ]:
print_closest_words("rainha")

princesa 29.976177
monarca 30.001593
príncipes 32.25395
príncipe 32.314346
reis 32.578526


In [ ]:
print_closest_words("rei")

reis 30.005331
príncipe 33.991837
imperador 34.584774
rainha 37.146923


In [ ]:
print_closest_words("chefe")

chefes 24.67743
líder 28.122522
comandante 34.328247
líderes 35.4594


In [ ]:
print_closest_words("dermatologista")

erghbal 0.0
irã 0.0
iraniana 0.0
desembarcaram 0.0
unido 0.0


In [ ]:
print_closest_words("mulher")

mulheres 21.180138


In [ ]:
print_closest_words("homem")

In [ ]:
print_closest_words("esposa")

marido 26.94526
companheira 32.758205


In [ ]:
print_closest_words("tio")

tia 26.300518
sobrinho 29.369762
avó 33.929928
sobrinha 34.837
neta 36.117825


In [ ]:
print_closest_words("tia")

tio 26.300518
sobrinho 31.95798
avó 32.94361
neta 33.700134
sobrinha 33.874615


In [ ]:
print_closest_words("marido")

esposa 26.94526


In [ ]:
print_closest_words("chefe")

chefes 24.67743
líder 28.122522
comandante 34.328247
líderes 35.4594


In [ ]:
print_closest_words("legal")

legais 25.327862
ilegal 32.964043


In [ ]:
token_vecs_sum_t = torch.stack((token_vecs_sum))


In [ ]:
torch.save(token_vecs_sum_t, "/content/drive/MyDrive/curso_word2vec/BertEmbeddings/PreTrainedTorches.json")

In [ ]:
len(token_vecs_sum)

158368

In [ ]:
len(tokenized_text)

158368

In [ ]:
with open("/content/drive/MyDrive/curso_word2vec/BertEmbeddings/PreTrainedWords.json", "w") as f:
  f.write(json.dumps(tokenized_text))